# Install Prerequisites

### Remove existing CUDA installation and NVIDIA drivers

### Install specific CUDA

### Installation check

In [ ]:
!ls /usr/local/


bin    cuda	cuda-11.8  games	       include	lib64	   man	 share
colab  cuda-11	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [ ]:
!which nvcc


In [ ]:
%%writefile hello.cu

#include<stdio.h>
__global__ void hello(void)
{
    printf("GPU: Hello!\n");
}
int main(int argc,char **argv)
{
    printf("CPU: Hello!\n");
    hello<<<1,10>>>();
    cudaDeviceReset();
    return 0;
}

Overwriting hello.cu


In [ ]:
!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 hello.cu -o hello

# Method 2
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-11 /usr/local/cuda
!nvcc hello.cu -o hello

nvcc warning : The 'compute_35', 'compute_37', 'sm_35', and 'sm_37' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [ ]:
!./hello


CPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!


Install a jupyter extension

Load the plugin

# Run CUDA

### Run a print example

In [ ]:
%%writefile test.cu
#include <iostream>

int main() {
    std::cout << "This is from CUDA\n";
    return 0;
}

Writing test.cu


In [ ]:
!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 hello.cu -o hello

# Method 2
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-11 /usr/local/cuda
!nvcc test.cu -o hello
!./hello

nvcc warning : The 'compute_35', 'compute_37', 'sm_35', and 'sm_37' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
This is from CUDA


### Involved example

In [ ]:
%%writefile matrix.cu
#include <cstdio>
#include <iostream>

using namespace std;

__global__ void maxi(int* a, int* b, int n)
{
	int block = 256 * blockIdx.x;
	int max = 0;

	for (int i = block; i < min(256 + block, n); i++) {

		if (max < a[i]) {
			max = a[i];
		}
	}
	b[blockIdx.x] = max;
}

int main()
{

	int n;
	n = 3 >> 2;
	int a[n];

	for (int i = 0; i < n; i++) {
		a[i] = rand() % n;
		cout << a[i] << "\t";
	}

	cudaEvent_t start, end;
	int *ad, *bd;
	int size = n * sizeof(int);
	cudaMalloc(&ad, size);
	cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
	int grids = ceil(n * 1.0f / 256.0f);
	cudaMalloc(&bd, grids * sizeof(int));

	dim3 grid(grids, 1);
	dim3 block(1, 1);

	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while (n > 1) {
		maxi<<<grids, block>>>(ad, bd, n);
		n = ceil(n * 1.0f / 256.0f);
		cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time = 0;
	cudaEventElapsedTime(&time, start, end);

	int ans[2];
	cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);

	cout << "The maximum element is : " << ans[0] << endl;

	cout << "The time required : ";
	cout << time << endl;
}


Writing matrix.cu


In [ ]:
#!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 matrix.cu -o hello

# Method 2
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-11 /usr/local/cuda
!nvcc matrix.cu -o max


In [ ]:
!./max

The maximum element is : 0
The time required : 0.003232


### Example 1

In [ ]:
%%writefile ex.cu

#include <stdio.h>

// This is a special function that runs on the GPU (device) instead of the CPU (host)
__global__ void kernel() {
  printf("Hello world!\n");
}

int main() {
  // Invoke the kernel function on the GPU with one block of one thread
  kernel<<<1,1>>>();

  // Check for error codes (remember to do this for _every_ CUDA function)
  if(cudaDeviceSynchronize() != cudaSuccess) {
    fprintf(stderr, "CUDA Error: %s\n", cudaGetErrorString(cudaPeekAtLastError()));
  }
  return 0;
}

Hello world!



### Example 2

In [ ]:
%%writefile ex2.cu

#include <stdio.h>

// This kernel runs on the GPU and prints the thread's identifiers
__global__ void kernel() {
  printf("Hello from block %d thread %d\n", blockIdx.x, threadIdx.x);
}

int main() {
  // Launch the kernel on the GPU with four blocks of six threads each
  kernel<<<4,6>>>();

  // Check for CUDA errors
  if(cudaDeviceSynchronize() != cudaSuccess) {
    fprintf(stderr, "CUDA Error: %s\n", cudaGetErrorString(cudaPeekAtLastError()));
  }
  return 0;
}

Writing ex2.cu


In [ ]:
!nvcc ex2.cu -o ex2

In [ ]:
!./ex2

Hello from block 2 thread 0
Hello from block 2 thread 1
Hello from block 2 thread 2
Hello from block 2 thread 3
Hello from block 2 thread 4
Hello from block 2 thread 5
Hello from block 0 thread 0
Hello from block 0 thread 1
Hello from block 0 thread 2
Hello from block 0 thread 3
Hello from block 0 thread 4
Hello from block 0 thread 5
Hello from block 3 thread 0
Hello from block 3 thread 1
Hello from block 3 thread 2
Hello from block 3 thread 3
Hello from block 3 thread 4
Hello from block 3 thread 5
Hello from block 1 thread 0
Hello from block 1 thread 1
Hello from block 1 thread 2
Hello from block 1 thread 3
Hello from block 1 thread 4
Hello from block 1 thread 5


### Example 3

In [ ]:
%%cu

#include <stdint.h>
#include <stdio.h>

#define N 32
#define THREADS_PER_BLOCK 32

__global__ void saxpy(float a, float* x, float* y) {
  // Which index of the array should this thread use?
  size_t index = 20;

  // Compute a times x plus y for a specific index
  y[index] = a * x[index] + y[index];
}

int main() {
  // Allocate arrays for X and Y on the CPU. This memory is only usable on the CPU
  float* cpu_x = (float*)malloc(sizeof(float) * N);
  float* cpu_y = (float*)malloc(sizeof(float) * N);

  // Initialize X and Y
  int i;
  for(i=0; i<N; i++) {
    cpu_x[i] = (float)i;
    cpu_y[i] = 0.0;
  }

  // The gpu_x and gpu_y pointers will only be usable on the GPU (which uses separate memory)
  float* gpu_x;
  float* gpu_y;

  // Allocate space for the x array on the GPU
  if(cudaMalloc(&gpu_x, sizeof(float) * N) != cudaSuccess) {
    fprintf(stderr, "Failed to allocate X array on GPU\n");
    exit(2);
  }

  // Allocate space for the y array on the GPU
  if(cudaMalloc(&gpu_y, sizeof(float) * N) != cudaSuccess) {
    fprintf(stderr, "Failed to allocate Y array on GPU\n");
    exit(2);
  }

  // Copy the cpu's x array to the gpu with cudaMemcpy
  if(cudaMemcpy(gpu_x, cpu_x, sizeof(float) * N, cudaMemcpyHostToDevice) != cudaSuccess) {
    fprintf(stderr, "Failed to copy X to the GPU\n");
  }

  // Copy the cpu's y array to the gpu with cudaMemcpy
  if(cudaMemcpy(gpu_y, cpu_y, sizeof(float) * N, cudaMemcpyHostToDevice) != cudaSuccess) {
    fprintf(stderr, "Failed to copy Y to the GPU\n");
  }

  // Calculate the number of blocks to run, rounding up to include all threads
  size_t blocks = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

  // Run the saxpy kernel
  saxpy<<<blocks, THREADS_PER_BLOCK>>>(0.5, gpu_x, gpu_y);

  // Wait for the kernel to finish
  if(cudaDeviceSynchronize() != cudaSuccess) {
    fprintf(stderr, "CUDA Error: %s\n", cudaGetErrorString(cudaPeekAtLastError()));
  }

  // Copy the y array back from the gpu to the cpu
  if(cudaMemcpy(cpu_y, gpu_y, sizeof(float) * N, cudaMemcpyDeviceToHost) != cudaSuccess) {
    fprintf(stderr, "Failed to copy Y from the GPU\n");
  }

  // Print the updated y array
  for(i=0; i<N; i++) {
    printf("%d: %f\n", i, cpu_y[i]);
  }

  cudaFree(gpu_x);
  cudaFree(gpu_y);
  free(cpu_x);
  free(cpu_y);

  return 0;
}


0: 0.000000
1: 0.000000
2: 0.000000
3: 0.000000
4: 0.000000
5: 0.000000
6: 0.000000
7: 0.000000
8: 0.000000
9: 0.000000
10: 0.000000
11: 0.000000
12: 0.000000
13: 0.000000
14: 0.000000
15: 0.000000
16: 0.000000
17: 0.000000
18: 0.000000
19: 0.000000
20: 10.000000
21: 0.000000
22: 0.000000
23: 0.000000
24: 0.000000
25: 0.000000
26: 0.000000
27: 0.000000
28: 0.000000
29: 0.000000
30: 0.000000
31: 0.000000



In [ ]:
%%writefile ntt.cu




#include <stdlib.h>
#include <random>

unsigned modpow64(unsigned a, unsigned b, unsigned mod)  // calculates (<a> ** <b>) mod <mod>
{
    unsigned res = 1;

    if (1 & b)
        res = a;

    while (b != 0)
    {
        b = b >> 1;
        unsigned long long t64 = (unsigned long long)a * a;
        a = t64 % mod;
        if (b & 1)
        {
            unsigned long long r64 = (unsigned long long)a * res;
            res = r64 % mod;
        }

    }
    return res;
}

unsigned long long bitReverse(unsigned long long a, int bit_length)  // reverses the bits for twiddle factor calculation
{
    unsigned long long res = 0;

    for (int i = 0; i < bit_length; i++)
    {
        res <<= 1;
        res = (a & 1) | res;
        a >>= 1;
    }

    return res;
}

std::random_device dev;  // uniformly distributed integer random number generator that produces non-deterministic random numbers
std::mt19937_64 rng(dev());  // pseudo-random generator of 64 bits with a state size of 19937 bits

void randomArray64(unsigned a[], int n, unsigned q)
{
    std::uniform_int_distribution<unsigned> randnum(0, q - 1);  // uniformly distributed random integers on the closed interval [a, b] according to discrete probability

    for (int i = 0; i < n; i++)
    {
        a[i] = randnum(rng);
    }
}

void fillTablePsi64(unsigned psi, unsigned q, unsigned psiinv, unsigned psiTable[], unsigned psiinvTable[], unsigned int n)  // twiddle factors computation
{
    for (int i = 0; i < n; i++)
    {
        psiTable[i] = modpow64(psi, bitReverse(i, log2(n)), q);
        psiinvTable[i] = modpow64(psiinv, bitReverse(i, log2(n)), q);
    }
}


#include "cuda_runtime.h"
#include "device_launch_parameters.h"

__device__ __forceinline__ void singleBarrett(unsigned long long& a, unsigned& q, unsigned& mu, int& qbit)  // ??
{
    unsigned long long rx;
    rx = a >> (qbit - 2);
    rx *= mu;
    rx >>= qbit + 2;
    rx *= q;
    a -= rx;

    a -= q * (a >= q);
}

__global__ void CTBasedNTTInnerSingle(unsigned a[], unsigned q, unsigned mu, int qbit, unsigned psi_powers[])
{
    int local_tid = threadIdx.x;

    extern __shared__ unsigned shared_array[];

    #pragma unroll
    for (int iteration_num = 0; iteration_num < 2; iteration_num++)
    {  // copying to shared memory from global
        int global_tid = local_tid + iteration_num * 1024;
        shared_array[global_tid] = a[global_tid + blockIdx.x * 2048];
    }

    #pragma unroll
    for (int length = 1; length < 2048; length *= 2)
    {  // iterations for ntt
        int step = (2048 / length) / 2;
        int psi_step = local_tid / step;
        int target_index = psi_step * step * 2 + local_tid % step;;

        psi_step = (local_tid + blockIdx.x * 1024) / step;

        unsigned psi = psi_powers[length + psi_step];

        unsigned first_target_value = shared_array[target_index];
        unsigned long long temp_storage = shared_array[target_index + step];  // this is for eliminating the possibility of overflow

        temp_storage *= psi;

        singleBarrett(temp_storage, q, mu, qbit);
        unsigned second_target_value = temp_storage;

        unsigned target_result = first_target_value + second_target_value;

        target_result -= q * (target_result >= q);

        shared_array[target_index] = target_result;

        first_target_value += q * (first_target_value < second_target_value);

        shared_array[target_index + step] = first_target_value - second_target_value;

        __syncthreads();
    }

    #pragma unroll
    for (int iteration_num = 0; iteration_num < 2; iteration_num++)
    {  // copying back to global from shared
        int global_tid = local_tid + iteration_num * 1024;
        a[global_tid + blockIdx.x * 2048] = shared_array[global_tid];
    }

}

__global__ void GSBasedINTTInnerSingle(unsigned a[], unsigned q, unsigned mu, int qbit, unsigned psiinv_powers[])
{
    int local_tid = threadIdx.x;

    extern __shared__ unsigned shared_array[];

    unsigned q2 = (q + 1) >> 1;

    #pragma unroll
    for (int iteration_num = 0; iteration_num < 2; iteration_num++)
    {  // copying to shared memory from global
        int global_tid = local_tid + iteration_num * 1024;
        shared_array[global_tid] = a[global_tid + blockIdx.x * 2048];
    }

    __syncthreads();


    for (int length = 1024; length >= 1; length /= 2)
    {  // iterations for intt
        int step = (2048 / length) / 2;

        int psi_step = local_tid / step;
        int target_index = psi_step * step * 2 + local_tid % step;

        psi_step = (local_tid + blockIdx.x * 1024) / step;

        unsigned psiinv = psiinv_powers[length + psi_step];

        unsigned first_target_value = shared_array[target_index];
        unsigned second_target_value = shared_array[target_index + step];

        unsigned target_result = first_target_value + second_target_value;

        target_result -= q * (target_result >= q);

        shared_array[target_index] = (target_result >> 1) + q2 * (target_result & 1);

        first_target_value += q * (first_target_value < second_target_value);

        unsigned long long temp_storage = first_target_value - second_target_value;

        temp_storage *= psiinv;

        singleBarrett(temp_storage, q, mu, qbit);

        unsigned temp_storage_low = temp_storage;

        shared_array[target_index + step] = (temp_storage_low >> 1) + q2 * (temp_storage_low & 1);


        __syncthreads();
    }


    for (int iteration_num = 0; iteration_num < 2; iteration_num++)
    {  // copying back to global from shared
        int global_tid = local_tid + iteration_num * 1024;
        a[global_tid + blockIdx.x * 2048] = shared_array[global_tid];
    }
}
#include <iostream>
using std::cout;  // yes we are that lazy
using std::endl;  // :)



#define check 1 // set to 0 if there is no need to check for the correctness of operations

int main()
{
    unsigned n = 2048;

    int size_array = sizeof(unsigned) * n;
    int size = sizeof(unsigned);

    unsigned q = 536608769, psi = 284166, psiinv = 208001377;  // parameter initialization
    unsigned int q_bit = 29;

    /****************************************************************
    BEGIN
    cudamalloc, memcpy, etc... for gpu
    */

    unsigned* psiTable = (unsigned*)malloc(size_array);
    unsigned* psiinvTable = (unsigned*)malloc(size_array);
    fillTablePsi64(psi, q, psiinv, psiTable, psiinvTable, n); //gel psi psi

    //copy powers of psi and psi inverse tables to device
    unsigned* psi_powers, * psiinv_powers;

    cudaMalloc(&psi_powers, size_array);
    cudaMalloc(&psiinv_powers, size_array);

    cudaMemcpy(psi_powers, psiTable, size_array, cudaMemcpyHostToDevice);
    cudaMemcpy(psiinv_powers, psiinvTable, size_array, cudaMemcpyHostToDevice);

    // we print these because we forgot them every time :)
    cout << "n = " << n << endl;
    cout << "q = " << q << endl;
    cout << "Psi = " << psi << endl;
    cout << "Psi Inverse = " << psiinv << endl;

    //generate parameters for barrett
    unsigned int bit_length = q_bit;
    double mu1 = powl(2, 2 * bit_length);
    unsigned mu = mu1 / q;

    unsigned* a;
    cudaMallocHost(&a, sizeof(unsigned) * n);
    randomArray64(a, n, q); //fill array with random numbers between 0 and q - 1

    unsigned* res_a;
    cudaMallocHost(&res_a, sizeof(unsigned) * n);

    unsigned* d_a;
    cudaMalloc(&d_a, size_array);

    cudaMemcpyAsync(d_a, a, size_array, cudaMemcpyHostToDevice, 0);

    /*
    END
    cudamalloc, memcpy, etc... for gpu
    ****************************************************************/


    /****************************************************************
    BEGIN
    Kernel Calls
    */
    CTBasedNTTInnerSingle<<<1, 1024, 2048 * sizeof(unsigned), 0>>>(d_a, q, mu, bit_length, psi_powers);
    GSBasedINTTInnerSingle<<<1, 1024, 2048 * sizeof(unsigned), 0>>>(d_a, q, mu, bit_length, psiinv_powers);
    /*
    END
    Kernel Calls
    ****************************************************************/

    cudaMemcpyAsync(res_a, d_a, size_array, cudaMemcpyDeviceToHost, 0);  // do this in async

    cudaDeviceSynchronize();  // CPU being a gentleman, and waiting for GPU to finish it's job

    bool correct = 1;
    if (check) //check the correctness of results
    {
        for (int i = 0; i < n; i++)
        {
           cout<<a[i]<<"= " <<res_a[i]<<endl;
            if (a[i] != res_a[i])
            {

                correct = 0;
                break;
            }
        }
    }

    if (correct)
        cout << "\nNTT and INTT are working correctly." << endl;
    else
        cout << "\nNTT and INTT are not working correctly." << endl;

    cudaFreeHost(a); cudaFreeHost(res_a);
    cudaFree(d_a);
    return 0;
}

Overwriting ntt.cu


In [ ]:
!nvcc ntt.cu -o ntt

ntt.cu(210): warning #177-D: variable "size" was declared but never referenced



In [ ]:
!./ntt

n = 2048
q = 536608769
Psi = 284166
Psi Inverse = 208001377
106199071= 106199071
180441029= 180441029
77230074= 77230074
487817230= 487817230
439270709= 439270709
45867569= 45867569
427137691= 427137691
348643319= 348643319
221202315= 221202315
181788275= 181788275
112994477= 112994477
146213987= 146213987
488580377= 488580377
31417826= 31417826
208670881= 208670881
429712878= 429712878
422552964= 422552964
21826136= 21826136
509395787= 509395787
399729926= 399729926
508317566= 508317566
116579739= 116579739
193165072= 193165072
285052389= 285052389
467604871= 467604871
383575669= 383575669
291060241= 291060241
246387898= 246387898
236774349= 236774349
356884042= 356884042
6966046= 6966046
395292500= 395292500
176037907= 176037907
264320145= 264320145
302455295= 302455295
254230120= 254230120
114234957= 114234957
431036179= 431036179
189604955= 189604955
1235482= 1235482
240824777= 240824777
320612824= 320612824
78820505= 78820505
338692707= 338692707
363489996= 363489996
77722353= 777

In [1]:
import math
import random
from sympy import isprime
def isInteger( M):
        return type(M).__name__ == 'int'

def isPrime( M):
    assert isInteger(M), 'Not an integer.'
    return isprime(M)

    # factorize algorithm
    # complexity is O(N^(1/2))
    # not used
def factorize( N):
    factors = []
    for factor in range(1, int(math.sqrt(N) + 1)):
        if N % factor == 0:
            if factor != 1:
                factors.append(factor)
                factors.append(int(N / factor))
    factors.sort()
    return factors

    # modular expoential algorithm
    # complexity is O(log N)
def modExponent( base, power, M):
    result = 1
    power = int(power)
    base = base % M
    while power > 0:
        if power & 1:
            result = (result * base) % M
        base = (base * base) % M
        power = power >> 1
    return result

    # calculate x^(-1) mod M
def modInv( x, M):
    t, new_t, r, new_r = 0, 1, M, x

    while new_r != 0:
        quotient = int(r / new_r)
        tmp_new_t = new_t
        tmp_t = t
        tmp_new_r = new_r
        tmp_r = r
        t, new_t = new_t, (t - quotient * new_t)
        r, new_r = new_r, (r % new_r)
    if r > 1:
        return "x is not invertible."
    if t < 0:
        t = t + M
    return t

    # check if r^k = 1 (mod M), k<N
def existSmallN( r, M, N):
    for k in range(2, N):
        if modExponent(r, k, M) == 1:
            return True
    return False
def NthRootOfUnity( M, N):
        assert isPrime(M), 'Not a prime.'  # modulus should be a prime
        assert (M - 1) % N == 0, 'N cannot divide phi(M)'
        phi_M = M - 1
        while True:
            alpha = random.randrange(1, M)
            beta = modExponent(alpha, phi_M / N, M)
            # check if beta can be k th root of unity, k<N
            if not existSmallN(beta, M, N):
                return int(beta)

def isNthRootOfUnity( M, N, beta):
        return modExponent(beta, N, M) == 1

In [2]:
M = 2**64-2**32+1
N = 1024
r = NthRootOfUnity(M, N)
if isNthRootOfUnity(M, N, r):
    print("%d to power of %d is congruent to 1 modulo %d" % (r, N, M))
else:
    print("%d to power of %d is not congruent to 1 modulo %d" % (r, N, M))
w = NthRootOfUnity(M, N)
print(w)
inv_w =modInv(w, M)
print("inverse w is",inv_w)
inv_N =modInv(N, M)
print("inverse n is",inv_N)

14056801952326137183 to power of 1024 is congruent to 1 modulo 18446744069414584321
9535690687442338791
inverse w is 5486745524883165993
inverse n is 18428729670909296641


In [ ]:
from sympy.ntheory import isprime, primitive_root
import math
def gen_omegas(n, q):
    # Generate an omega: g^k (mod q) for a generator of the field, g.
    print("started\n")
    g = primitive_root(q)
    print(g)
    k = (q - 1) // n
    print(k)
    omega = (g ** k) % q
    assert 0 <= omega and omega < q

    # Generate pre-computed omega array (also from Shunning).
    omegas = [1]
    for i in range(n):
        omegas.append((omegas[i] * omega) % q)
    for i in range(n):
        assert (omegas[n - i] * omegas[i]) % q == 1
    omegas = omegas[:n]  # Drop the last, needless value.

    return omegas


def inversed(omegas, q):
    def multiplicative_inverse(a, q):
        # We want to find `i` such that `(x * i) mod q == 1`,
        # which which is guaranteed if `x` is a unit of the
        # multiplicative group under `q`.
        #
        # Reference:
        # https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm
        prime = q
        Y = 0
        X = 1
        if a == 1:
            return 0
        while a > 1:
            quotient = a // q
            t = q
            q = a % q
            a = t
            t = Y
            Y = X - quotient * Y
            X = t
        return X if X >= 0 else X + prime

    return [multiplicative_inverse(x, q) for x in omegas]

In [5]:
q=2**64-2**32+1
n=1024
omegas=gen_omegas(n,q)
print(omegas)

started
7
18014398505287680


KeyboardInterrupt: 

In [ ]:
inv_omegas=inversed(omegas, q)
print(inv_omegas)